In [ ]:
import numpy as np 
import math
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import csv
import math
from matplotlib.patches import Rectangle

In [ ]:
constellations=["starlink","kuiper","oneweb","iridium"]
policy="P_OPT"

In [ ]:
# gs_type = "single"
label = "distributed"
single_upbound=[2000,10000,20000,30000]
result={}
name_s=['SpaceCore','NTN','SkyCore','DPCM','Tiansuan']

In [18]:
distributed_file="data/pairs.npy"
pair=np.array(np.load(distributed_file,allow_pickle=True))
print(np.shape(pair))

(10000,)


In [19]:
cycle=100
def cal_handover():
    r=[]
    for t in range(1,cycle):
        r_in=[]
        for i in range(len(ue_access)):
            if ue_access[i][t-1] != ue_access[i][t]:
                r_in.append(i)
        r.append(r_in)
    return r
handover_result={}
for constellation in constellations:
    path="data/"
    if constellation=="starlink":
        path+="starlink_72_22/"
    elif constellation=="kuiper":
        path+="kuiper/"
    elif constellation=="oneweb":
        path+="OneWeb-1week/"
    elif constellation=="iridium":
        path+="Iridium_1week/"
    ue_access_file=path+constellation+"_ue_access_"+policy+".npy"
    ue_access=np.array(np.load(ue_access_file,allow_pickle=True))
    ho=cal_handover()
    print(np.shape(ho))
    handover_result[constellation]=ho

C:\Users\DELL\AppData\Roaming\Python\Python36\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


(99,)
(99,)
(99,)
(99,)


In [25]:
#mobility
pdu_gsl=[[0,0],[3,40],[0,0],[0,0],[0,0]]
mobility_gsl=[[[0,0],[2,17],[0,0],[5,7],[5,7]],[[0,0],[1,0],[0,0],[1,1],[1,1]]]
gsl_mobility_signaling={}
for constellation in constellations:
    path="data/"
    if constellation=="starlink":
        path+="starlink_72_22/"
    elif constellation=="kuiper":
        path+="kuiper/"
    elif constellation=="oneweb":
        path+="OneWeb-1week/"
    elif constellation=="iridium":
        path+="Iridium_1week/"
    ue_access_file=path+constellation+"_ue_access_"+policy+".npy"
    ue_access=np.array(np.load(ue_access_file,allow_pickle=True))
    bound_signaling=[]
    for b_index in range(len(single_upbound)):    
        ratio_file="data/processed_data/"+constellation+"_per_user_ratio"+str(single_upbound[b_index])+".npy"
        ratio=np.array(np.load(ratio_file,allow_pickle=True))
        s_signaling=[]
        for s_index in range(len(name_s)):
            time_signaling=[]
            for c in range(len(handover_result[constellation])):
                gs_signaling=[0.0]*39
                for user in handover_result[constellation][c]:
                    cur_sat=ue_access[user][c]-1
                    cur_gs=pair[user]
                    r=ratio[c][cur_sat]
                    gs_signaling[cur_gs]+=(r*(mobility_gsl[0][s_index][0]+mobility_gsl[1][s_index][0]))
                time_signaling.append(gs_signaling)
            s_signaling.append(time_signaling)
        bound_signaling.append(s_signaling)
    print(np.shape(bound_signaling))
    gsl_mobility_signaling[constellation]=bound_signaling
    

(4, 5, 99, 39)
(4, 5, 99, 39)
(4, 5, 99, 39)
(4, 5, 99, 39)


In [26]:
#pdu
pdu_gsl=[[0,0],[3,40],[0,0],[0,0],[0,0]]
mobility_gsl=[[[0,0],[2,17],[0,0],[5,7],[5,7]],[[0,0],[1,0],[0,0],[1,1],[1,1]]]
gsl_session_signaling={}
for constellation in constellations:
    path="data/"
    if constellation=="starlink":
        path+="starlink_72_22/"
    elif constellation=="kuiper":
        path+="kuiper/"
    elif constellation=="oneweb":
        path+="OneWeb-1week/"
    elif constellation=="iridium":
        path+="Iridium_1week/"
    ue_access_file=path+constellation+"_ue_access_"+policy+".npy"
    ue_access=np.array(np.load(ue_access_file,allow_pickle=True))
    bound_signaling=[]
    for b_index in range(len(single_upbound)):    
        ratio_file="data/processed_data/"+constellation+"_per_user_ratio"+str(single_upbound[b_index])+".npy"
        ratio=np.array(np.load(ratio_file,allow_pickle=True))
        s_signaling=[]
        for s_index in range(len(name_s)):
            time_signaling=[]
            for c in range(len(handover_result[constellation])):
                gs_signaling=[0.0]*39
                if c%2==0:
                    for user in range(len(ue_access)):
                        cur_sat=ue_access[user][c]-1
                        cur_gs=pair[user]
                        r=ratio[c+1][cur_sat]
                        gs_signaling[cur_gs]+=(r*pdu_gsl[s_index][0])
                time_signaling.append(gs_signaling)
            s_signaling.append(time_signaling)
        bound_signaling.append(s_signaling)
    print(np.shape(bound_signaling))
    gsl_session_signaling[constellation]=bound_signaling
    

(4, 5, 99, 39)
(4, 5, 99, 39)
(4, 5, 99, 39)
(4, 5, 99, 39)


In [42]:
np.save("data/gsl_session_signaling.npy",gsl_session_signaling)
np.save("data/gsl_mobility_signaling.npy",gsl_mobility_signaling)